# SQL WINDOW Functions On Olympics Data

**Question 1**

The Summer Olympics dataset contains the results of the games between 1896 and 2012. The first Summer Olympics were held in 1896, the second in 1900, and so on. Find out, in which year the 13th Summer Olympics were held?

In [1]:
SELECT *
FROM (
	SELECT
		year,
		ROW_NUMBER() OVER() AS row_n
	FROM (
		SELECT DISTINCT(year)
		FROM medals.summer_medals
		ORDER BY Year ASC
	) AS years
	ORDER BY Year ASC
) AS final
WHERE row_n = 13;

,year,row_n
0,1956,13


**Question 2**

Rank the athletes based on the number of medals he or she has earned in the Olympics

In [2]:
WITH athlete_medals AS (
  SELECT
    -- Count the number of medals each athlete has earned
    athlete,
    COUNT(*) AS medals
  FROM medals.summer_medals
  GROUP BY Athlete)

SELECT
  athlete,
  ROW_NUMBER() OVER (ORDER BY medals DESC) AS rank
FROM athlete_medals
ORDER BY medals DESC;

,athlete,rank
0,PHELPS Michael,1
1,LATYNINA Larisa,2
2,ANDRIANOV Nikolay,3
3,MANGIAROTTI Edoardo,4
4,ONO Takashi,5
...,...,...
22757,TOMAN Ladislav,22758
22758,OUTTERIDGE Nathan,22759
22759,LISKOVA Hana,22760
22760,POLENOVA Elena,22761


**Question 3**

Determine if a country was the reigning champion during a particular year in the Men's 69KG weightlifting competition.

In [3]:
WITH weightlifting_gold AS (
	SELECT
    	-- Returning each year's champions' countries
    	year,
    	country AS champion
  	FROM medals.summer_medals
  	WHERE
    	discipline = 'Weightlifting' AND
    	event = '69KG' AND
		gender = 'Men' AND
    	medal = 'Gold')

SELECT *,
	CASE WHEN champion = last_champion THEN 'Yes'
	ELSE 'No' END AS reigning_champions
FROM (
	SELECT
  		year, champion,
  		-- Fetch the previous year's champion
  		LAG(champion) OVER
    		(ORDER BY year ASC) AS last_champion
	FROM weightlifting_gold
	ORDER BY Year ASC
) AS final;

,year,champion,last_champion,reigning_champions
0,2000,BUL,None,No
1,2004,CHN,BUL,No
2,2008,CHN,CHN,Yes
3,2012,CHN,CHN,Yes


**Question 4**

Determine if a country was the reigning champion during a particular year in the 100m and 10000m competitions. Split the table into partitions to avoid having a champion from one gender and event appear as the previous champion of another gender and event.

In [4]:
WITH athletics_gold AS (
	SELECT DISTINCT
		gender, year, event, country
	FROM medals.summer_medals
	WHERE
		year >= 2000
		AND discipline = 'Athletics'
		AND event IN ('100M', '10000M')
		AND medal = 'Gold')

SELECT
	gender, year, event,
	country AS champion,
  	-- Fetch the previous year's champion by gender and event
	LAG(country) OVER (PARTITION BY gender, event
					ORDER BY year ASC) AS last_champion
FROM athletics_gold
ORDER BY event ASC, gender ASC, year ASC;

,gender,year,event,champion,last_champion
0,Men,2000,10000M,ETH,None
1,Men,2004,10000M,ETH,ETH
2,Men,2008,10000M,ETH,ETH
3,Men,2012,10000M,GBR,ETH
4,Women,2000,10000M,ETH,None
5,Women,2004,10000M,CHN,ETH
6,Women,2008,10000M,ETH,CHN
7,Women,2012,10000M,ETH,ETH
8,Men,2000,100M,USA,None
9,Men,2004,100M,USA,USA


**Question 5**

Rank the athletes in each country by the count of medals they've earned -- the higher the count, the higher the rank -- without skipping numbers in case of identical values.

In [5]:
WITH athlete_medals AS (
	SELECT
		country,
		athlete,
		COUNT(*) AS medals
	FROM medals.summer_medals
	WHERE
		country IN ('JPN', 'KOR')
		AND year >= 2000
	GROUP BY country, athlete
	HAVING COUNT(*) > 1
)

SELECT
	country,
	-- Rank athletes in each country by the medals they've won
	Athlete,
	medals,
	DENSE_RANK() OVER (PARTITION BY country
					ORDER BY medals DESC) AS rank_n
FROM athlete_medals
ORDER BY country ASC, rank_n ASC;

,country,athlete,medals,rank_n
0,JPN,KITAJIMA Kosuke,7,1
1,JPN,UCHIMURA Kohei,5,2
2,JPN,TAKEDA Miho,4,3
3,JPN,TACHIBANA Miya,4,3
4,JPN,IRIE Ryosuke,3,4
...,...,...,...,...
69,KOR,OH Sangeun,2,4
70,KOR,LEE Bo Na,2,4
71,KOR,OH Seongok,2,4
72,KOR,LEE Dong Soo,2,4


**Question 6**

Split the athletes into top, middle, and bottom thirds based on their count of medals.

In [6]:
WITH athlete_medals AS (
	SELECT
		athlete,
		COUNT(*) AS medals
	FROM medals.summer_medals
	GROUP BY athlete
	HAVING COUNT(*) > 1
)
  
SELECT
	athlete,
	medals,
	-- Spliting athletes into thirds by their earned medals
	NTILE(3) OVER(ORDER BY medals DESC) AS third
FROM athlete_medals
ORDER BY medals DESC, athlete ASC;

,athlete,medals,third
0,PHELPS Michael,22,1
1,LATYNINA Larisa,18,1
2,ANDRIANOV Nikolay,15,1
3,MANGIAROTTI Edoardo,13,1
4,ONO Takashi,13,1
...,...,...,...
5261,ZÜRNER Albert,2,1
5262,ZVEREVA Ellina,2,2
5263,ZWERVER Ronald,2,3
5264,ZWOLLE Hendrik Jan,2,2


**Question 7**

Continuing on the above question, return the average of each third.

In [7]:
WITH athlete_medals AS (
	SELECT
		athlete,
		COUNT(*) AS medals
	FROM medals.summer_medals
	GROUP BY athlete
	HAVING COUNT(*) > 1
),
thirds AS (
	SELECT
		athlete,
    	medals,
    	NTILE(3) OVER (ORDER BY medals DESC) AS third
	FROM athlete_medals
)
  
SELECT
	-- Get the average medals earned in each third
	third,
	AVG(medals) AS avg_medals
FROM thirds
GROUP BY third
ORDER BY third ASC;

,third,avg_medals
0,1,3.786446
1,2,2.000000
2,3,2.000000


**Question 8**

Return the year, country, medals, and the maximum medals earned so far for each country. Comparing the current number of medals with maximum medals earned by a country helps in determining whether a country broke its own medals record.

In [8]:
WITH country_medals AS (
	SELECT
		year,
		country,
		COUNT(*) AS medals
	FROM medals.summer_medals
	WHERE
		country IN ('CHN', 'KOR', 'JPN')
		AND medal = 'Gold' AND Year >= 2000
	GROUP BY year, country
)

SELECT
	-- Return the max medals earned so far per country
	year,
	country,
	medals,
	MAX(medals) OVER (PARTITION BY country
					ORDER BY year ASC) AS max_medals
FROM country_medals
ORDER BY country ASC, year ASC;

,year,country,medals,max_medals
0,2000,CHN,39,39
1,2004,CHN,52,52
2,2008,CHN,74,74
3,2012,CHN,56,74
4,2000,JPN,5,5
5,2004,JPN,21,21
6,2008,JPN,23,23
7,2012,JPN,7,23
8,2000,KOR,12,12
9,2004,KOR,14,14


**Question 9**

Return the year, medals earned, and the maximum medals earned, comparing only the current year and the next year for Scandinavian Countries.
NOTE : We do not want the result for each scandinavian country, but for the scandinavian countries as a whole.

In [9]:
WITH scandinavian_medals AS (
	SELECT
		year,
		COUNT(*) AS medals
	FROM medals.summer_medals
	WHERE
		country IN ('DEN', 'NOR', 'FIN', 'SWE', 'ISL')
		AND medal = 'Gold'
	GROUP BY year
)

SELECT
	year,
	medals,
	-- Get the max of the current and next years'  medals
	MAX(medals) OVER (ORDER BY year ASC
					ROWS BETWEEN CURRENT ROW
					AND 1 FOLLOWING) AS max_medals
FROM scandinavian_medals
ORDER BY year ASC;

,year,medals,max_medals
0,1896,1,1
1,1900,1,77
2,1908,77,141
3,1912,141,159
4,1920,159,159
5,1924,48,48
6,1928,24,24
7,1932,17,17
8,1936,15,54
9,1948,54,54


**Question 10**

Calculate the 3-year moving average of medals earned by Russia.

In [10]:
WITH russian_medals AS (
	SELECT
		year,
		COUNT(*) AS medals
	FROM medals.summer_medals
	WHERE
		country = 'RUS'
		AND medal = 'Gold'
		AND year >= 1980
	GROUP BY year
)

SELECT
	year, medals,
	--- Calculate the 3-year moving average of medals earned
	AVG(medals) OVER
		(ORDER BY year ASC
    	ROWS BETWEEN
		2 PRECEDING AND CURRENT ROW) AS medals_ma
FROM russian_medals
ORDER BY year ASC;

,year,medals,medals_ma
0,1996,36,36.000000
1,2000,66,51.000000
2,2004,47,49.666667
3,2008,43,52.000000
4,2012,47,45.666667


**Question 11**

Calculate the 3-year moving sum of medals earned per country.

In [11]:
WITH country_medals AS (
	SELECT
		year,
		country,
		COUNT(*) AS medals
	FROM medals.summer_medals
	GROUP BY year, country
)

SELECT
	year, country, medals,
	-- Calculate each country's 3-game moving total
	SUM(medals) OVER
		(PARTITION BY country
		 ORDER BY year ASC
		 ROWS BETWEEN
		 2 PRECEDING AND CURRENT ROW) AS medals_ma
FROM country_medals
ORDER BY country ASC, year ASC;

,year,country,medals,medals_ma
0,2012,,4,4
1,2008,AFG,1,1
2,2012,AFG,1,2
3,1988,AHO,1,1
4,1984,ALG,2,2
...,...,...,...,...
1153,2004,ZIM,3,19
1154,2008,ZIM,4,23
1155,1896,ZZX,6,6
1156,1900,ZZX,34,40


**Question 12**

Let's look at three Scandinavian countries' earned gold medals per country and gender in the year 2004. Also calculate the Country-level subtotals to get the total medals earned for each country (in a single query).

NOTE : Turn all nulls into meaningful texts.

In [12]:
SELECT
	-- Replace the nulls in the columns with meaningful text
	COALESCE(country, 'All Countries') AS country,
	COALESCE(gender, 'All Genders') AS gender,
	COUNT(*) AS awards
FROM medals.summer_medals
WHERE
	year = 2004
	AND medal = 'Gold'
	AND country IN ('DEN', 'NOR', 'SWE')
-- Generate Country-level subtotals
GROUP BY ROLLUP(country, gender)
ORDER BY country ASC, gender ASC;

,country,gender,awards
0,All Countries,All Genders,29
1,DEN,All Genders,19
2,DEN,Men,4
3,DEN,Women,15
4,NOR,All Genders,5
5,NOR,Men,3
6,NOR,Women,2
7,SWE,All Genders,5
8,SWE,Men,4
9,SWE,Women,1


**Question 13**

Generate a break down of all medals awarded to Russia in the 2012 Olympic games per gender and medal type. Since the medals all belong to one country, Russia, it makes sense to generate all possible subtotals (Gender- and Medal-level subtotals), as well as a grand total.

NOTE : Turn all nulls into meaningful texts.

In [13]:
-- Count the medals per gender and medal type
SELECT
	COALESCE(gender, 'All Genders') AS gender,
	COALESCE(medal, 'All Medals') AS medals,
	COUNT(*) AS awards
FROM medals.summer_medals
WHERE
	year = 2012
	AND country = 'RUS'
-- Get all possible group-level subtotals
GROUP BY CUBE(gender, medals)
ORDER BY gender ASC, medals ASC;

,gender,medals,awards
0,All Genders,Bronze,51
1,All Genders,Gold,47
2,All Genders,Silver,32
3,All Genders,None,130
4,Men,Bronze,34
5,Men,Gold,23
6,Men,Silver,7
7,Men,None,64
8,Women,Bronze,17
9,Women,Gold,24


**Question 14**

After ranking each country in the 2000 Olympics by gold medals awarded, return the top 3 countries in one row, as a comma-separated string.

In [14]:
WITH country_medals AS (
	SELECT
		country,
		COUNT(*) AS medals
	FROM medals.summer_medals
	WHERE year = 2000
		AND medal = 'Gold'
	GROUP BY country
),
country_ranks AS (
	SELECT
		country,
		-- Rank countries by the medals awarded
		RANK() OVER (ORDER BY medals DESC) AS rank
	FROM country_medals
	ORDER BY rank ASC
)

-- Compress the countries column
SELECT STRING_AGG(Country, ', ')
FROM country_ranks
-- Select only the top three ranks
WHERE rank <= 3;

,string_agg
0,"USA, RUS, AUS"


**Question 15**

Retrieve the names of athletes, events, and years. Also, include an additional column, previous_winner, indicating the athlete who previously won a gold medal in the same event. Filter the results to include only gold medalists.

In [15]:
SELECT
	athlete,
	event,
	year,
	LAG(athlete) OVER(PARTITION BY event
					 ORDER BY year) AS previous_winner
FROM medals.summer_medals
WHERE medal = 'Gold';

,athlete,event,year,previous_winner
0,LORD CURRIE Lorne Campbell,0.5-1T,1900,None
1,GRETTON John H.,0.5-1T,1900,LORD CURRIE Lorne Campbell
2,HOPE Linton,0.5-1T,1900,GRETTON John H.
3,KOLEHMAINEN Hannes,10000M,1912,None
4,NURMI Paavo,10000M,1920,KOLEHMAINEN Hannes
...,...,...,...,...
10481,AYTON Sarah,Yngling - Keelboat,2004,WEBB Sarah
10482,WEBB Sarah,Yngling - Keelboat,2008,AYTON Sarah
10483,AYTON Sarah,Yngling - Keelboat,2008,WEBB Sarah
10484,WILSON Pippa,Yngling - Keelboat,2008,AYTON Sarah


**Question 16**

Provide the year, the cumulative count of medalists for each year, and the ongoing total count of medalists in Summer Olympics history. Arrange the results in ascending order based on the year.

In [16]:
SELECT
	year,
	COUNT(*) AS total_winners,
	SUM(COUNT(*)) OVER(ORDER BY year ASC
					ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_sum
FROM medals.summer_medals
GROUP BY year
ORDER BY year;

,year,total_winners,running_sum
0,1896,151,151
1,1900,512,663
2,1904,470,1133
3,1908,804,1937
4,1912,885,2822
5,1920,1298,4120
6,1924,884,5004
7,1928,710,5714
8,1932,615,6329
9,1936,875,7204


**Question 17**

Provide a list of countries, along with the corresponding years and the count of gold medals achieved. Restrict the results to the years 2004, 2008, and 2012. Additionally, display the total number of gold medals earned by each country across these three years.

In [17]:
SELECT
	country,
	COALESCE(CAST(year AS VARCHAR), 'Combined Years') AS year,
	COUNT(medal) AS gold_medals
FROM medals.summer_medals
WHERE year IN ('2004', '2008', '2012')
	AND medal = 'Gold'
GROUP BY country, ROLLUP(year)
ORDER BY country, year;

,country,year,gold_medals
0,,2012,2
1,,Combined Years,2
2,ALG,2012,1
3,ALG,Combined Years,1
4,ARG,2004,26
...,...,...,...
237,VEN,2012,1
238,VEN,Combined Years,1
239,ZIM,2004,1
240,ZIM,2008,1
